In [96]:
from arcgis.gis import GIS
import json

In [2]:
gis = GIS(profile = "eowilson")

In [3]:
import re
def create_strict_reg_exp(to_search):
    try:
        reg_exp = f"^{to_search}$"
        #logging.info(f"regular expression is: {reg_exp}")
    except:
        print("There was a problem with the string.")
    return reg_exp

In [4]:
def findItemGetID(csvName, gis):
    try:
        searched_item = gis.content.search(csvName, item_type = "Feature Layer")
        for i in searched_item:
            reg_exp = create_strict_reg_exp(csvName)
            if re.search(reg_exp, i.title)!= None:    
                #logging.info(f"{csvName} has the id: {i.id}")
                return i.id
    except:
        print("There was a problem finding the item")

In [5]:
def getFlayerFromID(item_id):
    item = gis.content.get(item_id)
    flayer = item.layers[0]
    return flayer

In [10]:
import pandas as pd
def getSDFfromFlayer(flayer):
    sdf = pd.DataFrame.spatial.from_layer(flayer)
    return sdf

# Get the item and get an SDF

In [7]:
item_id = findItemGetID("gadm_centroid", gis)

In [8]:
flayer = getFlayerFromID(item_id)

In [11]:
sdf = getSDFfromFlayer(flayer)

# Get below and above countries for each field in challenges

In [15]:
fields = ['GID_0',	'NAME_0', 'Area','GNI_PPP','Population2016', 'prop_protected', 'prop_hm_very_high', 'protection_needed', 'total_endemic','N_SPECIES', 'SPI']

In [16]:
df = sdf[fields]

In [89]:
filter_fields = ['Area','GNI_PPP','Population2016', 'prop_protected', 'prop_hm_very_high', 'protection_needed', 'total_endemic','N_SPECIES', 'SPI']

In [107]:
nber_index = 5
max_index = len(df_sort.index) - 1
df_sort = df
new_fields = []
for field in filter_fields:
    df_sort = df_sort.sort_values(by = [field]).reset_index(drop=True)
    collapse_list = []
    for index, i in df_sort.iterrows():
        country_gid = df_sort.GID_0[index]
        above_index = index - nber_index
        below_index = index + nber_index + 1
        if above_index < 0:
            below_index = nber_index * 2 
            above_index = 0
        if below_index > max_index:
            above_index = max_index - (nber_index * 2)
            below_index = max_index

        sub_pd = df_sort.GID_0[above_index:below_index]
        val_list = sub_pd.values.tolist()
        collapse_list.append(json.dumps(val_list))

        #val_list_rem = val_list.remove(country_gid)
    filter_field = f"filter_{field}"
    new_fields.append(filter_field)
    df_sort[filter_field] = collapse_list

In [136]:
similar_list = []
for index, i in df_sort.iterrows():
    filter_dict = i[new_fields].to_dict()
    similar_list.append(json.loads(json.dumps(filter_dict)))    
df_sort['filter_similar'] = similar_list